In [1]:
import requests
import socket
import time

if not locals().get('camera'): camera = None

class CIDR4:
    def __init__(self, cidr: str) -> None:
        self.cidr = cidr
        self.ip_str, netmask = cidr.split('/')
        try:
            netmask_int = int(netmask)
            self.netmask_list = self.get_netmask(netmask_int)
        except ValueError as e:
            self.netmask_list = [ int(x) for x in netmask.split('.') ]
        self.ip_list = [ int(x) for x in self.ip_str.split('.') ]
        self.ip1_list = [ x & y for x, y in zip(self.ip_list, self.netmask_list)]
        self.ip2_list = [ x | y for x, y in zip(self.ip_list, self.netmask_list)]

    def get_netmask(self, netmask_int = 0) -> list:
        netmask_list = list()
        for i in range(4):
            x = 0
            for j in range(8):
                x <<= 1
                if netmask_int > 0:
                    netmask_int -= 1
                    x |= 1
            netmask_list.append(x)
        return netmask_list
    
    def merge_ip(self, cidr) -> str:
        if isinstance(cidr, str):
            return self.merge_ip(CIDR4(cidr))
        else: # isinstance(cidr, CIDR4)
            merged_ip_str_list = [ str(x | y ^ z) for x, y, z in zip(self.ip1_list, cidr.ip2_list, cidr.netmask_list) ]
            merged_ip_str = '.'.join(merged_ip_str_list)
            return merged_ip_str

api_url = "https://mirror.ghproxy.com/https://raw.githubusercontent.com/mzbgf/esp32-cam/main/api"
local_cidr = '192.168.0.164/24'

def get_server():
    request_times = 0
    while request_times < 3:
        req = requests.get(url=api_url + "/static/server_info.json", timeout=5)
        if req.ok:
            break
    return req.json()

# server_info = get_server()
server_info = {'type': 'tcp', 'cidr': '192.168.31.11/255.255.255.0', 'port': 9999}

print(server_info)

{'type': 'tcp', 'cidr': '192.168.31.11/255.255.255.0', 'port': 9999}


In [2]:
server_type = server_info['type']
server_cidr = server_info['cidr']
server_port = server_info['port']

local_server_ip = CIDR4(local_cidr).merge_ip(server_cidr)
server_addr = (local_server_ip, server_port)
print(server_addr)

('192.168.0.11', 9999)
